<a href="https://colab.research.google.com/github/Greeshmalikithareddy2004/AIML_Projects_and_Labs/blob/main/Copy_of_AIML_Module_1_Lab_2_Machine_learning_terms_and_metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##AIML Module 1 - Lab 2
# Machine learning terms and metrics


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [3]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-3-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [4]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [5]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [6]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [7]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [8]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [9]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [10]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [11]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [12]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [13]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [17]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [16]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33824843372875724
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


**Does averaging the validation accuracy across multiple splits give more consistent results?**

Averaging validation accuracy across multiple splits, often referred to as cross-validation, can indeed provide more consistent and reliable results compared to using a single validation split. Cross-validation is a common technique in machine learning and statistics used to assess the performance of a model and mitigate the potential impact of randomness in data splitting.

Here's why averaging validation accuracy across multiple splits is beneficial:

Reduced Variance: By performing multiple splits of your data into training and validation sets and then calculating the validation accuracy for each split, you reduce the influence of a single random split. This helps in obtaining a more stable estimate of your model's performance.

Better Generalization: Cross-validation allows you to assess how well your model generalizes to different subsets of your data. This is particularly important when you have a limited amount of data, as it helps ensure that your model isn't overfitting to a specific validation set.




**Does it give more accurate estimate of test accuracy?**

Cross-validation, which involves averaging validation accuracy across multiple splits of your data, provides a more reliable estimate of your model's generalization performance compared to a single validation split. However, it's important to note that the estimated test accuracy obtained through cross-validation is still an approximation of the true test accuracy that you would obtain on unseen data.

Here's why cross-validation provides a more accurate estimate of test accuracy compared to a single validation split:

Reduced Overfitting to the Validation Set: With a single validation split, there's a risk of overfitting to that specific validation set, which can lead to an overly optimistic estimate of your model's performance. Cross-validation helps mitigate this risk by averaging over multiple validation sets, making it less likely that your model's performance is overly optimistic.

More Comprehensive Assessment: Cross-validation allows you to evaluate your model's performance across multiple different subsets of the data, which provides a more comprehensive view of how well your model generalizes to unseen data. This can help you uncover issues with your model that might not be apparent with a single validation split.




**What is the effect of the numbers of iterations on the estimate? Do we get a better estimate with high iteration**

The number of iterations or folds in cross-validation can have an impact on the accuracy of the estimate of your model's performance. In general, using a higher number of iterations (folds) can provide a better estimate, but there are trade-offs to consider:

More Stable Estimate: Increasing the number of iterations in cross-validation tends to yield a more stable and reliable estimate of your model's performance. This is because you're averaging the results from multiple validation sets, reducing the influence of randomness in the data split.

Reduced Bias: A larger number of iterations can help reduce bias in your estimate. With a single split, your model might perform unusually well or poorly on that specific validation set due to random chance. With more iterations, the impact of such outliers is mitigated.

Increased Computational Cost: Performing cross-validation with a high number of iterations can be computationally expensive, especially if your dataset is large or if your model is complex. Each iteration requires training and evaluating the model, which can be time-consuming.

Diminishing Returns: There are diminishing returns in terms of the accuracy of the estimate as you increase the number of iterations. Beyond a certain point, the improvement in accuracy becomes marginal, while the computational cost continues to increase.

**Consider the results you got for the previous questions. Can we deal with a very small dataset or validation dataset by increasing the iterations?**

Mitigating Variability: With a small dataset, the risk of obtaining unreliable performance estimates is higher because the choice of a single validation set can have a significant impact on the results. By increasing the number of iterations in cross-validation, you reduce the impact of this variability and obtain a more stable estimate of your model's performance.

Improving Reliability: When the validation dataset is small, there's a greater chance that it may not be representative of the overall data distribution. Using more iterations in cross-validation allows you to sample from different subsets of the data, increasing the chances of capturing the data's full diversity.

Trade-offs with Computational Cost: While increasing the number of iterations is beneficial for improving the estimate's reliability, it comes with an increased computational cost. Training and evaluating the model multiple times can be time-consuming, especially if your dataset is very small, and you may need to balance computational resources with the desire for a more stable estimate.

Limited Data: Cross-validation can only work with the data you have. If your dataset is extremely small, increasing the number of iterations can only do so much to alleviate the limitations associated with having insufficient data for training and validation.

Generalization Challenges: Very small datasets may still pose challenges for model generalization, even with cross-validation. In such cases, it's essential to be cautious about interpreting performance estimates. Extremely small datasets may not provide a reliable indication of how well the model will perform on unseen data.